In [1]:
# Cell 1: Environment Setup
# =================================
import os
import sys
import json
import re
from pathlib import Path
import numpy as np
import torch
import transformers
from datasets import Dataset, load_dataset, DatasetDict, concatenate_datasets
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    pipeline
)

# Set environment variables
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["NO_TF"] = "1"  # Prevent TensorFlow import issues

# Verify installations
print("\n=== Core Package Versions ===")
print(f"Python: {sys.version}")
print(f"NumPy: {np.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

2025-06-05 04:28:33.396239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749097713.646506      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749097713.722304      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



=== Core Package Versions ===
Python: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
NumPy: 1.26.4
PyTorch: 2.6.0+cu124
Transformers: 4.51.3
CUDA available: False


In [2]:
# Cell 2: Model Loading
# ============================
MODEL_NAME = "gpt2"

def print_memory():
    """Memory usage diagnostics"""
    import psutil
    ram = psutil.virtual_memory()
    print(f"RAM: {ram.percent:.1f}% ({ram.used/1024**3:.1f}/{ram.total/1024**3:.1f}GB)")

def load_model(model_name):
    print(f"\n=== Loading Model: {model_name} ===")
    print_memory()
    
    device = "cpu"
    torch_dtype = torch.float32
    
    try:
        print("Attempting standard CPU load...")
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map=None,
            torch_dtype=torch_dtype
        ).to(device)
        print("\n✅ Model loaded successfully on CPU!")
        return model
    except Exception as e:
        print(f"\n❌ Standard load failed: {str(e)}")
        raise RuntimeError("Unable to load model on CPU")

model = load_model(MODEL_NAME)


=== Loading Model: gpt2 ===
RAM: 6.0% (1.4/31.4GB)
Attempting standard CPU load...


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


✅ Model loaded successfully on CPU!


In [3]:
# Cell 3: Tokenizer Setup
# ==============================
def load_tokenizer(model_name):
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            padding_side="right"
        )
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        print("Tokenizer loaded successfully")
        return tokenizer
    except Exception as e:
        print(f"Tokenizer loading failed: {str(e)}")
        raise

tokenizer = load_tokenizer(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Tokenizer loaded successfully


In [4]:
# Cell 4: Dataset Preparation
# ==================================
def extract_qna_from_notebook(notebook_path):
    """Extract Q&A pairs from Jupyter notebook cells"""
    try:
        with open(notebook_path, 'r', encoding='utf-8') as f:
            notebook = json.load(f)
        
        qna_pairs = []
        qna_patterns = [
            r"(?:^|\n)(Q|Question)[:：]?\s*(.+?)\s*(A|Answer)[:：]?\s*(.+?)(?=\n\s*(?:Q|Question|$))",
            r"##\s*(.+?)\s*\n([\s\S]+?)\n(?:##|\Z)",
            r"\"\"\"\s*Q:(.+?)\nA:(.+?)\"\"\"",
            r"<qa>\n<q>(.+?)</q>\n<a>(.+?)</a>\n</qa>"
        ]
        
        for cell in notebook['cells']:
            if cell['cell_type'] == 'markdown':
                text = ''.join(cell['source'])
                
                for pattern in qna_patterns:
                    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
                    if matches:
                        for match in matches:
                            if len(match) == 4:
                                question = match[1].strip()
                                answer = match[3].strip()
                            elif len(match) == 2:
                                question = match[0].strip()
                                answer = match[1].strip()
                            else:
                                continue
                                
                            if question and answer:
                                qna_pairs.append(f"Q: {question}\nA: {answer}")
        
        return qna_pairs
    
    except Exception as e:
        print(f"Error processing notebook: {str(e)}")
        return []

def create_fallback_dataset():
    """Create fallback dataset using reliable public datasets"""
    crypto_qa = [
        "Q: What is Bitcoin? A: Bitcoin is a decentralized digital currency.",
        "Q: What is Ethereum? A: Ethereum is a blockchain platform with smart contract functionality.",
        "Q: What is a blockchain? A: A distributed ledger technology recording transactions across networks.",
        "Q: What is a crypto wallet? A: Software/hardware storing private keys to manage cryptocurrencies.",
        "Q: What is mining in cryptocurrency? A: The process of validating transactions and creating new coins.",
        "Q: What is DeFi? A: Decentralized finance using blockchain without traditional intermediaries.",
        "Q: What is an NFT? A: Non-fungible token representing unique digital ownership.",
        "Q: What is a smart contract? A: Self-executing contracts with terms directly written into code.",
        "Q: What is proof-of-stake? A: Consensus mechanism where validators stake crypto to secure network.",
        "Q: What is a DAO? A: Decentralized Autonomous Organization governed by smart contracts."
    ]
    return Dataset.from_dict({"text": crypto_qa})

def prepare_dataset(file_path="/kaggle/input/database4", max_samples=1000):
    try:
        print(f"\n🔍 Searching for dataset at: {file_path}")
        data_path = Path(file_path)
        
        if not data_path.exists():
            raise FileNotFoundError(f"Dataset path not found: {data_path}")
        
        # 1. Check for specific notebook file
        specific_notebook = data_path / "database-0604.ipynb"
        if specific_notebook.exists():
            print(f"✅ Found specific notebook file: {specific_notebook}")
            qna_pairs = extract_qna_from_notebook(specific_notebook)
            
            if qna_pairs:
                print(f"Extracted {len(qna_pairs)} Q&A pairs from notebook")
                return Dataset.from_dict({"text": qna_pairs[:max_samples]})
            else:
                print("No Q&A pairs found in notebook, checking other sources")
        
        # 2. Look for other notebook files
        notebook_files = list(data_path.rglob('*.ipynb'))
        if notebook_files:
            print(f"Found {len(notebook_files)} notebook files")
            for notebook_path in notebook_files:
                if notebook_path == specific_notebook:
                    continue
                print(f"Processing notebook: {notebook_path.name}")
                qna_pairs = extract_qna_from_notebook(notebook_path)
                if qna_pairs:
                    print(f"Extracted {len(qna_pairs)} Q&A pairs from {notebook_path.name}")
                    return Dataset.from_dict({"text": qna_pairs[:max_samples]})
        
        # 3. Try loading Q&A pairs from JSON
        qna_files = list(data_path.rglob("*qna*.json")) + list(data_path.rglob("*qa*.json"))
        if qna_files:
            print(f"Found {len(qna_files)} Q&A files")
            dataset = load_dataset('json', data_files=[str(f) for f in qna_files], 
                                  split=f'train[:{max_samples}]')
            return dataset
        
        # 4. Fallback to raw text extraction
        text_files = list(data_path.rglob('*.txt')) + list(data_path.rglob('*.md'))
        if text_files:
            print(f"Found {len(text_files)} text files")
            texts = []
            for file in text_files[:10]:
                try:
                    with open(file, 'r', encoding='utf-8') as f:
                        texts.append(f.read())
                except Exception as e:
                    print(f"Skipped file {file.name}: {str(e)}")
            return Dataset.from_dict({"text": texts[:max_samples]})
        
        raise ValueError("No usable data files found")
    
    except Exception as e:
        print(f"\n❌ Dataset loading failed: {str(e)}")
        print("Creating fallback dataset...")
        return create_fallback_dataset()

def safe_tokenize(examples):
    """Tokenization with error handling"""
    try:
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            max_length=256,
            padding="max_length",
            return_tensors="pt"
        )
        return {
            "input_ids": tokenized["input_ids"].tolist(),
            "attention_mask": tokenized["attention_mask"].tolist(),
            "labels": tokenized["input_ids"].tolist()
        }
    except Exception as e:
        print(f"Tokenization error: {str(e)}")
        return {
            "input_ids": [[0]*256],
            "attention_mask": [[1]*256],
            "labels": [[0]*256]
        }

# Process dataset
print("\n=== Starting Data Processing ===")
dataset = prepare_dataset("/kaggle/input/database4")

# Show sample data
print("\nSample data:")
for i in range(min(3, len(dataset))):
    print(f"\nSample {i+1}:")
    print(dataset[i]['text'][:200] + "...")

# Tokenize dataset
tokenized_dataset = dataset.map(
    safe_tokenize, 
    batched=True, 
    batch_size=4
)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Split dataset
split_datasets = tokenized_dataset.train_test_split(test_size=0.2)
tokenized_dataset = DatasetDict({
    "train": split_datasets["train"],
    "test": split_datasets["test"]
})
print(f"✅ Dataset split: {len(tokenized_dataset['train'])} train, {len(tokenized_dataset['test'])} test")


=== Starting Data Processing ===

🔍 Searching for dataset at: /kaggle/input/database4
✅ Found specific notebook file: /kaggle/input/database4/database-0604.ipynb
No Q&A pairs found in notebook, checking other sources
Found 1 notebook files

❌ Dataset loading failed: No usable data files found
Creating fallback dataset...

Sample data:

Sample 1:
Q: What is Bitcoin? A: Bitcoin is a decentralized digital currency....

Sample 2:
Q: What is Ethereum? A: Ethereum is a blockchain platform with smart contract functionality....

Sample 3:
Q: What is a blockchain? A: A distributed ledger technology recording transactions across networks....


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

✅ Dataset split: 8 train, 2 test


In [6]:
# Cell 5: Training Configuration - FIXED
# =====================================
model.gradient_checkpointing_enable()

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["attn.c_attn", "attn.c_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir=f"./{MODEL_NAME}-crypto-expert",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    optim="adamw_torch",
    logging_steps=10,
    eval_strategy="epoch",  # Changed from evaluation_strategy to eval_strategy
    save_strategy="epoch",
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    use_cpu=True
)

# Prepare model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainable params: 884,736 || all params: 125,324,544 || trainable%: 0.7060


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [7]:
# Cell 6: Training Execution
# =================================
def train_model(model, tokenized_dataset, training_args):
    """Execute the training process"""
    model.config.use_cache = False
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        data_collator=data_collator
    )
    
    print("\n=== Starting Training ===")
    trainer.train()
    
    # Save model
    output_dir = training_args.output_dir
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"\n✅ Model saved to {output_dir}")
    return trainer

trainer = train_model(model, tokenized_dataset, training_args)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



=== Starting Training ===


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,3.724031
2,No log,3.720621
3,No log,3.718847



✅ Model saved to ./gpt2-crypto-expert


In [8]:
# Cell 7: Model Testing
# =================================
def test_model(model, tokenizer, test_prompts=None):
    """Test the trained model"""
    if test_prompts is None:
        test_prompts = [
            "What is a hardware wallet?",
            "Explain Proof of Work in blockchain",
            "How does cryptocurrency mining work?",
            "What are the benefits of decentralized finance?",
            "Describe how smart contracts work"
        ]
    
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device=-1  # Force CPU
    )
    
    print("\n=== Model Test Results ===")
    for prompt in test_prompts:
        print(f"\nPrompt: {prompt}")
        output = pipe(
            prompt,
            max_length=150,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            num_return_sequences=1
        )
        print("Response:", output[0]['generated_text'])

test_model(model, tokenizer)

Device set to use cpu
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCa


=== Model Test Results ===

Prompt: What is a hardware wallet?
Response: What is a hardware wallet?

An electronic wallet is an electronic device that can be used to create a digital wallet. It is used by the user to store, exchange and transfer funds between computers.

The user must not use the electronic wallet for any reason.

How can I use the wallet for my personal use?

The online wallet is a digital wallet that is a digital currency. It is used to create a digital wallet.

What is a digital wallet?

The digital wallet is an electronic device that allows the user to store and exchange bitcoins. The digital wallet is used by the user to store, exchange and transfer bitcoins.

How do I use it for business?

The online wallet

Prompt: Explain Proof of Work in blockchain
Response: Explain Proof of Work in blockchain technology

At the time, this was a relatively new concept and a very new and unique technology, but with the emergence of the Ethereum blockchain, it became a common s

In [9]:
# Cell 7: Enhanced Model Saving with Shard Support
# ===============================================

# Add missing import
from typing import Optional

def save_model_artifacts(
    model, 
    tokenizer, 
    training_args: Optional[TrainingArguments] = None, 
    output_dir: str = "/kaggle/working/gpt2-lora-trained"
) -> str:
    """
    Save all model artifacts with comprehensive verification.
    Handles both single-file and sharded model formats.
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n💾 Saving model artifacts to: {output_dir}")
    
    # For LoRA models - DON'T merge adapters before saving
    # We want to save the adapter separately
    print("💽 Saving model and adapter...")
    
    # Save the entire model (base model + adapter)
    model.save_pretrained(
        output_dir,
        safe_serialization=True,
        state_dict=model.state_dict()  # Save the complete state including LoRA
    )
    
    # Save tokenizer
    print("🔤 Saving tokenizer...")
    tokenizer.save_pretrained(output_dir)
    
    # Save training arguments if provided
    if training_args is not None:
        print("📝 Saving training arguments...")
        try:
            args_path = os.path.join(output_dir, "training_args.json")
            if hasattr(training_args, 'to_dict'):
                with open(args_path, "w") as f:
                    json.dump(training_args.to_dict(), f, indent=2)
            elif hasattr(training_args, 'to_json_string'):
                with open(args_path, "w") as f:
                    f.write(training_args.to_json_string())
            else:
                print("⚠️ Warning: TrainingArguments has no serialization method")
        except Exception as e:
            print(f"⚠️ Warning: Failed to save training args - {str(e)}")
    
    # Verify the adapter files were saved
    required_files = ['adapter_config.json', 'adapter_model.safetensors']
    missing_files = []
    for file in required_files:
        if not os.path.exists(os.path.join(output_dir, file)):
            missing_files.append(file)
    
    if missing_files:
        print(f"\n⚠️ Warning: Missing adapter files: {missing_files}")
        print("Trying alternative save method...")
        # Explicitly save the adapter
        model.save_pretrained(
            output_dir,
            safe_serialization=True,
            adapter_only=True  # This ensures adapter files are saved
        )
    
    print("\n🔍 Verifying saved files:")
    for file in os.listdir(output_dir):
        size = os.path.getsize(os.path.join(output_dir, file)) / 1024
        print(f"- {file} ({size:.2f} KB)")
    
    return output_dir

In [12]:
# Cell 8: Robust Model Loading and Testing with PEFT support - COMPLETE
# ========================================================
from typing import Optional
from peft import PeftModel
from transformers import pipeline

def load_and_test_model(
    model_path: str = "/kaggle/working/gpt2-lora-trained", 
    max_length: int = 160,
    test_prompts: Optional[list] = None,
    is_peft_model: bool = True
):
    """
    Load and test a saved model with comprehensive error handling
    - Uses the same base model we trained on (GPT-2)
    - Handles both PEFT and regular model loading
    - Includes detailed error reporting
    """
    print(f"\n🔍 Preparing to load model from: {model_path}")
    
    # Verify model directory exists
    if not os.path.exists(model_path):
        raise ValueError(f"Model directory {model_path} does not exist")
    
    # Show directory contents for debugging
    print("\n📂 Model directory contents:")
    for f in sorted(os.listdir(model_path)):
        size = os.path.getsize(os.path.join(model_path, f)) / 1024
        print(f"- {f} ({size:.2f} KB)")
    
    try:
        print("\n🔄 Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        
        print("\n🔄 Loading model...")
        if is_peft_model:
            # First check if we have adapter files
            adapter_files = [
                f for f in os.listdir(model_path) 
                if f.startswith('adapter_') or f == 'adapter_config.json'
            ]
            
            if not adapter_files:
                print("⚠️ No adapter files found. Loading as regular model.")
                model = AutoModelForCausalLM.from_pretrained(
                    model_path,
                    torch_dtype=torch.float32
                )
            else:
                print(f"Found adapter files: {adapter_files}")
                # Load base model first - using the same base model we trained on (GPT-2)
                base_model = AutoModelForCausalLM.from_pretrained(
                    MODEL_NAME,  # Using our original MODEL_NAME (gpt2)
                    torch_dtype=torch.float32
                )
                
                # Then load the PEFT adapter
                model = PeftModel.from_pretrained(
                    base_model,
                    model_path
                )
                
                # Merge and unload for inference
                model = model.merge_and_unload()
        else:
            # For regular models
            model = AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch.float32
            )
            
        print("\n🎉 Model loaded successfully!")
        model.eval()  # Set to evaluation mode
        
        # Default test prompts if none provided
        if test_prompts is None:
            test_prompts = [
                "What is hardware wallet?",
                "What is Proof of Work (PoW)?",
                "What is cryptography?",
                "What is Peer-to-Peer (P2P)?",
                "What is block chain?",
                "What is private key?"
            ]
        
        # Create pipeline
        print("\n🚀 Creating text generation pipeline...")
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            device=-1,  # Force CPU usage
            pad_token_id=tokenizer.eos_token_id  # Ensure proper padding
        )
        
        # Run tests
        print("\n🧪 Running generation tests...")
        results = []
        for i, prompt in enumerate(test_prompts, 1):
            print(f"\n🔹 Test {i}: {prompt}")
            try:
                output = pipe(
                    prompt,
                    max_length=max_length,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    num_return_sequences=1,
                    repetition_penalty=1.2
                )
                response = output[0]['generated_text']
                print("💬 Response:", response)
                results.append({"prompt": prompt, "response": response})
            except Exception as e:
                print(f"⚠️ Error generating response: {str(e)}")
                results.append({"prompt": prompt, "error": str(e)})
        
        return model, tokenizer, results
        
    except Exception as e:
        print(f"\n❌ Critical error loading model: {str(e)}")
        print("\n🛠️ Debugging info:")
        print(f"- Path: {os.path.abspath(model_path)}")
        print(f"- Directory exists: {os.path.exists(model_path)}")
        if os.path.exists(model_path):
            print("- Contents:", os.listdir(model_path))
        raise

def save_model_artifacts(
    model, 
    tokenizer, 
    training_args: Optional[TrainingArguments] = None, 
    output_dir: str = "/kaggle/working/gpt2-lora-trained"
) -> str:
    """
    Save all model artifacts with comprehensive verification
    """
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n💾 Saving model artifacts to: {output_dir}")
    
    # Save model and adapter
    print("💽 Saving model and adapter...")
    model.save_pretrained(
        output_dir,
        safe_serialization=True,
        state_dict=model.state_dict()
    )
    
    # Save tokenizer
    print("🔤 Saving tokenizer...")
    tokenizer.save_pretrained(output_dir)
    
    # Save training arguments if provided
    if training_args is not None:
        print("📝 Saving training arguments...")
        try:
            args_path = os.path.join(output_dir, "training_args.json")
            if hasattr(training_args, 'to_dict'):
                with open(args_path, "w") as f:
                    json.dump(training_args.to_dict(), f, indent=2)
            else:
                print("⚠️ Warning: TrainingArguments has no serialization method")
        except Exception as e:
            print(f"⚠️ Warning: Failed to save training args - {str(e)}")
    
    # Verify saved files
    print("\n🔍 Verifying saved files:")
    for file in os.listdir(output_dir):
        size = os.path.getsize(os.path.join(output_dir, file)) / 1024
        print(f"- {file} ({size:.2f} KB)")
    
    return output_dir

In [13]:
# Main execution
if __name__ == "__main__":
    model_path = "/kaggle/working/gpt2-lora-trained"
    
    # Save model artifacts
    save_model_artifacts(model, tokenizer, training_args)
    
    # Load with explicit path and PEFT flag
    print("\n=== Testing with default prompts ===")
    model, tokenizer, default_results = load_and_test_model(
        model_path, 
        is_peft_model=True
    )
    
    # Test with custom prompts
    custom_prompts = [
        "What is software wallet, and what's the difference between hardware and software wallet?",
        "What is PoW?",
        "Explain PoW in 1 sentence.",
        "Describe the key features of PoW using 3 words.",
        "What is PoM? Is it something related to cryptography?",
        "What is a cryptographic product?",
        "What is P2P?",
        "What is block chain?",
        "What is public key, and what's the difference between private and public key?"
    ]
    
    print("\n=== Testing with custom prompts ===")
    model, tokenizer, custom_results = load_and_test_model(
        model_path, 
        test_prompts=custom_prompts, 
        is_peft_model=True
    )


💾 Saving model artifacts to: /kaggle/working/gpt2-lora-trained
💽 Saving model and adapter...
🔤 Saving tokenizer...
📝 Saving training arguments...

🔍 Verifying saved files:
- tokenizer.json (3474.39 KB)
- adapter_config.json (0.69 KB)
- merges.txt (445.62 KB)
- README.md (4.96 KB)
- vocab.json (779.45 KB)
- special_tokens_map.json (0.13 KB)
- training_args.json (3.95 KB)
- adapter_model.safetensors (3461.98 KB)
- tokenizer_config.json (0.52 KB)

=== Testing with default prompts ===

🔍 Preparing to load model from: /kaggle/working/gpt2-lora-trained

📂 Model directory contents:
- README.md (4.96 KB)
- adapter_config.json (0.69 KB)
- adapter_model.safetensors (3461.98 KB)
- merges.txt (445.62 KB)
- special_tokens_map.json (0.13 KB)
- tokenizer.json (3474.39 KB)
- tokenizer_config.json (0.52 KB)
- training_args.json (3.95 KB)
- vocab.json (779.45 KB)

🔄 Loading tokenizer...

🔄 Loading model...
Found adapter files: ['adapter_config.json', 'adapter_model.safetensors']


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



🎉 Model loaded successfully!

🚀 Creating text generation pipeline...

🧪 Running generation tests...

🔹 Test 1: What is hardware wallet?
💬 Response: What is hardware wallet?
The software to store your money in a digital currency called Bitcoin. The technology, which was developed by Chinese-based developer Guo Huijun and his team at the University of Zurich (UZ), has been used as an alternative payment system for many years but its popularity continues growing rapidly because it enables users to pay with their smartphone or tablet without ever having even had access into what's known as "virtual wallets". It can be downloaded from any major online retailer such Google Play Store - not including eBay where there are no physical stores selling bitcoins yet -- although they have also received support through Bitpay: https://bitcointalk.org/index...

🔹 Test 2: What is Proof of Work (PoW)?
💬 Response: What is Proof of Work (PoW)?
The PoS model uses a number of algorithms to determine if the

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



🎉 Model loaded successfully!

🚀 Creating text generation pipeline...

🧪 Running generation tests...

🔹 Test 1: What is software wallet, and what's the difference between hardware and software wallet?
💬 Response: What is software wallet, and what's the difference between hardware and software wallet?
It depends. Hardware wallets are more secure because they allow you to store your money in a safe place without having it be stored on an external hard drive or USB stick (or even outside of jail). Software-based systems that use "smart" technology such as Bluetooth make their transactions safer by making sure no data from any device ever gets lost while using them. That means anyone can easily change funds at will — especially if someone else has access for security reasons! You'll find out which methods work best when comparing different versions now with our guides here. It also allows us better control over how much I need my Bitcoins so we're able both keep track of all transaction fe